In [16]:
import os
import numpy as np
import rasterio
from rasterio.features import rasterize
from rasterio.plot import reshape_as_raster
from osgeo import gdal, ogr
import geopandas as gpd

In [17]:
def compute_aspect_slope(dem, transform, slope_units='degrees'):
    """
    Compute aspect and slope from DEM using numpy.
    """
    x, y = np.gradient(dem, transform[0], transform[4])  # Gradient in x (EW) and y (NS) directions
    slope = np.arctan(np.sqrt(x**2 + y**2))
    aspect = np.arctan2(-y, x)  # Aspect in radians (arctan2 handles 360° wrapping)
    
    # Convert slope to degrees or percent rise
    if slope_units == 'degrees':
        slope = np.degrees(slope)
    elif slope_units == 'percent':
        slope = np.tan(slope) * 100
    return aspect, slope

def compute_hillshade(dem, transform, azimuth=315, altitude=45):
    """
    Compute hillshade using numpy.
    """
    x, y = np.gradient(dem, transform[0], transform[4])
    slope = np.arctan(np.sqrt(x**2 + y**2))
    aspect = np.arctan2(-y, x)
    azimuth_rad = np.radians(azimuth)
    altitude_rad = np.radians(altitude)

    hillshade = (
        np.sin(altitude_rad) * np.cos(slope)
        + np.cos(altitude_rad) * np.sin(slope) * np.cos(azimuth_rad - aspect)
    )
    return hillshade * 255

def compute_curvature(dem, transform):
    """
    Compute curvature using numpy.
    """
    x, y = np.gradient(dem, transform[0], transform[4])
    xx, xy = np.gradient(x, transform[0], transform[4])
    yx, yy = np.gradient(y, transform[0], transform[4])
    curvature = xx + yy
    return curvature

def save_raster(output_path, data, template_raster):
    """
    Save numpy array as a GeoTIFF file.
    """
    with rasterio.open(
        output_path,
        "w",
        driver="GTiff",
        height=data.shape[0],
        width=data.shape[1],
        count=1,
        dtype=data.dtype,
        crs=template_raster.crs,
        transform=template_raster.transform,
    ) as dst:
        dst.write(data, 1)

def generate_rasters(input_dem, output_folder, slope_units='degrees'):
    """
    Generate aspect, slope, hillshade, and curvature rasters from an input DEM.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    with rasterio.open(input_dem) as src:
        dem = src.read(1).astype(float)
        transform = src.transform

        # Aspect and Slope
        aspect, slope = compute_aspect_slope(dem, transform, slope_units=slope_units)
        save_raster(os.path.join(output_folder, "aspect.tif"), aspect, src)
        save_raster(os.path.join(output_folder, "slope.tif"), slope, src)

        # Hillshade
        hillshade = compute_hillshade(dem, transform)
        save_raster(os.path.join(output_folder, "hillshade.tif"), hillshade.astype(np.uint8), src)

        # Curvature
        curvature = compute_curvature(dem, transform)
        save_raster(os.path.join(output_folder, "curvature.tif"), curvature, src)

    print(f"Rasters saved to {output_folder}")

def generate_dem_from_contours(contour_file, output_dem, cell_size, elev_field):
    """
    Generate a DEM from contour lines using GDAL with user-defined elevation field.
    """
    # Open contour file
    contour_ds = ogr.Open(contour_file)
    if not contour_ds:
        raise FileNotFoundError(f"Contour file not found: {contour_file}")
    
    contour_layer = contour_ds.GetLayer()
    extent = contour_layer.GetExtent()  # xmin, xmax, ymin, ymax
    
    # Create raster
    x_res = int((extent[1] - extent[0]) / cell_size)
    y_res = int((extent[3] - extent[2]) / cell_size)
    
    target_ds = gdal.GetDriverByName("GTiff").Create(
        output_dem, x_res, y_res, 1, gdal.GDT_Float32
    )
    target_ds.SetGeoTransform((extent[0], cell_size, 0, extent[3], 0, -cell_size))
    target_ds.SetProjection(contour_layer.GetSpatialRef().ExportToWkt())
    
    # Rasterize using the elevation field
    gdal.RasterizeLayer(target_ds, [1], contour_layer, options=[f"ATTRIBUTE={elev_field}"])
    target_ds = None
    print(f"DEM generated and saved to {output_dem}")
    return output_dem

In [24]:
# Initial question to define if a DEM will be generated or if it already exists

input_type = input("What is your input, 'contour lines' or 'DEM'? ").strip().lower()

What is your input, 'contour lines' or 'DEM'?  contour lines


In [26]:
# Conditions depending on the input

if input_type == 'dem':
    # User input for DEM processing
    input_dem = input("Enter the full path to the input DEM file (e.g., 'C:/data/dem.tif'): ").strip()
    output_folder = input("Enter the full path to the output folder (e.g., 'C:/data/output/'): ").strip()
    slope_units = input("Enter the slope units ('degrees' or 'percent'): ").strip().lower()

    # Validate slope units
    if slope_units not in ['degrees', 'percent']:
        print("Invalid slope_units input. Defaulting to 'degrees'.")
        slope_units = 'degrees'

    # Validate DEM file existence
    if not os.path.exists(input_dem):
        raise FileNotFoundError(f"The specified DEM file does not exist: {input_dem}")

    # Run raster generation
    generate_rasters(input_dem, output_folder, slope_units)

elif input_type == 'contour lines':
    # Contour lines handling
    contour_file = input("Enter the full path to the contour lines file (e.g., C:/data/contours.shp): ").strip()
    output_dem = input("Enter the full path to the output DEM file (e.g., C:/data/output_dem.tif): ").strip()
    cell_size = float(input("Enter the desired cell size for the DEM (e.g., 10.0): ").strip())
    
    # Load shapefile and display attributes
    try:
        gdf = gpd.read_file(contour_file)
        print("Shapefile opened successfully.")
        print(f"Layer name: {os.path.basename(contour_file)}")
        print(f"EPSG code: {gdf.crs}")
        print(f"Number of rows: {len(gdf)}")
        print(f"Geometry type: {gdf.geom_type.unique()}")
        print(f"Columns (fields) in the attribute table: {list(gdf.columns)}")

        # User selects the elevation field
        elev_field = input("Please type the name of the elevation field: ").strip()
        if elev_field not in gdf.columns:
            raise ValueError(f"Field '{elev_field}' not found in the attribute table.")

    except Exception as e:
        print(f"Error processing shapefile: {e}")
        raise

    # Generate DEM
    generated_dem = generate_dem_from_contours(contour_file, output_dem, cell_size, elev_field)

    # Proceed with terrain analysis
    output_folder = input("Enter the full path to the output folder (e.g., C:/data/output/): ").strip()
    slope_units = input("Enter the slope units ('degrees' or 'percent'): ").strip().lower()
    
    if slope_units not in ['degrees', 'percent']:
        print("Invalid slope_units input. Defaulting to 'degrees'.")
        slope_units = 'degrees'

    generate_rasters(generated_dem, output_folder, slope_units)

else:
    print("Invalid input. Please enter 'DEM' or 'contour lines'.")

Enter the full path to the contour lines file (e.g., 'C:/data/contours.shp'):  C:\Users\jpjmn\OneDrive\Documents\GIS\curvas_BLP\curvas_BLP.shp
Enter the full path to the output DEM file (e.g., 'C:/data/output_dem.tif'):  C:\Users\jpjmn\OneDrive\Documents\GIS\curvas_BLP\output_dem.tif
Enter the desired cell size for the DEM (e.g., 10.0):  10


Shapefile opened successfully.
Layer name: curvas_BLP.shp
EPSG code: EPSG:32613
Number of rows: 2787
Geometry type: ['LineString' 'MultiLineString']
Columns (fields) in the attribute table: ['Elevation', 'Elev', 'geometry']


Please type the name of the elevation field:  Elev


DEM generated and saved to C:\Users\jpjmn\OneDrive\Documents\GIS\curvas_BLP\output_dem.tif


Enter the full path to the output folder (e.g., 'C:/data/output/'):  C:\Users\jpjmn\OneDrive\Documents\GIS\rasters\
Enter the slope units ('degrees' or 'percent'):  degrees


Rasters saved to C:\Users\jpjmn\OneDrive\Documents\GIS\rasters\
